Step 1: download and convert dataset from CSV to Parquet

In [1]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq

In [2]:
import dask.dataframe as dd
import pandas as pd
import numpy as np
from dask.distributed import Client, LocalCluster
import dask
from dask.distributed import get_task_stream

In [3]:
import time

In [4]:
print('pandas version: %s' % pd.__version__)
print('numpy version: %s' % np.__version__)
print('dask version: %s' % dask.__version__)

pandas version: 1.3.4
numpy version: 1.20.3
dask version: 2021.10.0


https://docs.databricks.com/_static/notebooks/koalas-benchmark-distributed-execution.html?_ga=2.216403934.95291449.1648935555-599276868.1645477063

In [5]:
filename = "taxi_dataset.txt"

In [6]:
with open(filename) as file:
    csv_files = [line.rstrip() for line in file]
# only choose yellow taxis
yellow = list(filter(lambda x: "yellow" in x, csv_files))

In [7]:
# make the list small for now
yellow = ['https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2013-11.csv']

In [8]:
yellow

['https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2013-11.csv']

In [9]:
import re
pattern = '/[^\/]*\.csv$/gm'

for csv_url in yellow:
#     get the filename only so I can sort by month / year
    csv_name = re.findall(r"[^\/]*\.csv$",csv_url)
    df = dd.read_csv(csv_url,dtype={'tolls_amount': 'float64'})

    df = df.repartition(npartitions=4)
    df.to_parquet(f'./tmp/trip_data_{csv_name}', write_index=False)


/Users/mk/opt/anaconda3/lib/python3.9/site-packages/dask/dataframe/io/csv.py:125: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  df = pandas_read_text(
/Users/mk/opt/anaconda3/lib/python3.9/site-packages/dask/dataframe/io/csv.py:125: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  df = pandas_read_text(
/Users/mk/opt/anaconda3/lib/python3.9/site-packages/dask/dataframe/io/csv.py:125: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  df = pandas_read_text(


Read in parquet to dask

In [47]:
from dask.distributed import get_task_stream

client = Client()

{'description': {'filename': '', 'name': '', 'line_number': 0, 'line': ''},
 'children': {},
 'count': 0,
 'identifier': 'root'}

# set up testbench

In [35]:
def benchmark(f, df, benchmarks, task_name, **kwargs):
    """Benchmark the given function against the given DataFrame.
    
    Parameters
    ----------
    f: function to benchmark
    df: data frame
    benchmarks: container for benchmark results
    name: task name
    
    Returns
    -------
    Duration (in seconds) of the given operation
    """
    ret_benchmark_vals = {}
    with get_task_stream(plot='save', filename="task-stream.html") as ts:
        start_time = time.time()
        ret = f(df, **kwargs)
        ret_benchmark_vals['raw_duration'] = time.time() - start_time
        ret_benchmark_vals['history'] = ts.data
    benchmarks[task_name] = ret_benchmark_vals
    print(f"{task_name} took: {benchmarks[task_name].get('raw_duration')} seconds")
    return benchmarks[task_name].get("raw_duration")

In [31]:
import collections
dask_benchmarks = collections.defaultdict(dict)
# benchmarks = {"task1" : {"stat1": val, "stat2": val}}

# Define benchmark tasks

In [64]:
all_tasks = []

In [65]:
# sum, then means = simple mapreduce
def read_to_basic_ETL(df = None):
    df = dd.read_parquet(
    "./tmp/", 
    storage_options={"anon": True, 'use_ssl': True})    
    return (df.fare_amount + df.tip_amount).mean().compute()
all_tasks.append(read_to_basic_ETL)

In [66]:
# counts of values seen = simple map, groupby, reduce
def count_values(df):
    return df.fare_amount.value_counts().compute()
all_tasks.append(count_values)

In [67]:
# cpu heavy arithmetic : mapreduce
def complicated_arithmetic_operation(df):
    theta_1 = df.pickup_longitude
    phi_1 = df.pickup_latitude
    theta_2 = df.dropoff_longitude
    phi_2 = df.dropoff_latitude
    temp = (np.sin((theta_2-theta_1)/2*np.pi/180)**2
           + np.cos(theta_1*np.pi/180)*np.cos(theta_2*np.pi/180) * np.sin((phi_2-phi_1)/2*np.pi/180)**2)
    ret = 2 * np.arctan2(np.sqrt(temp), np.sqrt(1-temp))
    return ret.compute()
all_tasks.append(complicated_arithmetic_operation)

In [68]:
def groupby_statistics(df):
    return df.groupby(by='passenger_count').agg(
      {
        'total_amount': ['mean', 'std'], 
        'tip_amount': ['mean', 'std']
      }
    ).compute()
all_tasks.append(complicated_arithmetic_operation)

In [69]:
# join two datasets
def join_data(df):
    return dd.merge(df, other, left_index=True, right_index=True).compute()
all_tasks.append(complicated_arithmetic_operation)

# run the tasks

In [70]:
df = dd.read_parquet(
    "./tmp/", 
    storage_options={"anon": True, 'use_ssl': True})

In [71]:
for task in all_tasks:
    benchmark(task, df=df, benchmarks = dask_benchmarks, task_name = task.__name__)

read_to_basic_ETL took: 0.4521312713623047 seconds
count_values took: 0.34616804122924805 seconds
complicated_arithmetic_operation took: 3.579279661178589 seconds
complicated_arithmetic_operation took: 2.8689677715301514 seconds
complicated_arithmetic_operation took: 2.600781202316284 seconds
